# Equity of Induced Transit Trips

In this workbook we analyze how changes in travel time and demand brought on by the introduciton of additional transit infrastructure is distributed accross populations.

First, we set up our modules and load the appropriate data:

In [1]:
import os
import pandas as pd
import altair as alt
import numpy as np

# Replace with your data folder paths
mx_folder = r"data/matrices/input"
counts_folder = r"data/counts/input"
link_folder = r"data/counts/interim"
output_folder = r"data/counts/output"

As we load in our dataset, let's sum demand by each origin and display the total differences in transit trips made to learn about how much is induced:

In [8]:
dtypes = {'i':int, 'j':int, 'travel_time':float, 'demand':float}
mx_BAU = pd.read_csv(os.path.join(mx_folder, 'bau_times_flows.csv'), dtype=dtypes)[['i', 'demand']].groupby('i', as_index=False).sum()
mx_A = pd.read_csv(os.path.join(mx_folder, 'scenario_A_times_flows.csv'), dtype=dtypes)[['i', 'demand']].groupby('i', as_index=False).sum()
mx_B = pd.read_csv(os.path.join(mx_folder, 'scenario_B_times_flows.csv'), dtype=dtypes)[['i', 'demand']].groupby('i', as_index=False).sum()
mx_C = pd.read_csv(os.path.join(mx_folder, 'scenario_C_times_flows.csv'), dtype=dtypes)[['i', 'demand']].groupby('i', as_index=False).sum()

print("Scenario A:", (mx_A.demand.sum()-mx_BAU.demand.sum()), "extra demand")
print("Scenario B:", (mx_B.demand.sum()-mx_BAU.demand.sum()), "extra demand")
print("Scenario C:", (mx_C.demand.sum()-mx_BAU.demand.sum()), "extra demand")

Scenario A: 13733.152638852596 extra demand
Scenario B: 4025.5698905665195 extra demand
Scenario C: 2771.1263431145344 extra demand


Since we are using induced trips directly, we can join together our demand data for each scenario and calculate the deltas:

In [16]:
delta = pd.merge(mx_BAU[['i', 'demand']], mx_A[['i', 'demand']], on='i')
delta.columns = ['i', 'demand_BAU', 'demand_A']
delta = pd.merge(delta, mx_B[['i', 'demand']], on='i')
delta.columns = ['i', 'demand_BAU', 'demand_A', 'demand_B']
delta = pd.merge(delta, mx_C[['i', 'demand']], on='i')
delta.columns = ['i', 'demand_BAU', 'demand_A', 'demand_B', 'demand_C']

delta['delta_A'] = delta['demand_A'] - delta['demand_BAU']
delta['delta_B'] = delta['demand_B'] - delta['demand_BAU']
delta['delta_C'] = delta['demand_C'] - delta['demand_BAU']
delta.head()

# Write the file for mapping
delta.to_csv(os.path.join(output_folder, 'trips_added.csv'), index=False)

Finally, we can load in our demographic data and calculate the distributions of the score across various demographic groups

In [17]:
taz_da = pd.read_csv(os.path.join(link_folder, 'taz_da_link.csv'), dtype={'DAUID': int, 'taz_id': int, 'frac_da_in_taz': float})
da_demo = pd.read_csv(os.path.join(counts_folder, 'da_census_profile.csv'))
delta_demo = pd.merge(delta, taz_da, left_on='i', right_on='taz_id')
delta_demo = pd.merge(delta_demo, da_demo, on='DAUID')
delta_da = delta_demo[delta_demo.columns[-11:-1]].multiply(delta_demo['frac_da_in_taz'], axis="index")
delta_da.columns = [f"f_{c}" for c in delta_da.columns]
delta_demo = pd.concat([delta_demo[['delta_A', 'delta_B', 'delta_C']], delta_da], axis=1)

# Now we do the weighted summary
delta_demo['A_pop_2016'] = (delta_demo['f_pop_2016']/delta_demo['f_pop_2016'].sum())*delta_demo['delta_A']
delta_demo['A_vm_minority'] = (delta_demo['f_vm_minority']/delta_demo['f_vm_minority'].sum())*delta_demo['delta_A']
delta_demo['A_income_lim'] = (delta_demo['f_income_lim']/delta_demo['f_income_lim'].sum())*delta_demo['delta_A']
delta_demo['A_labour_unemployed'] = (delta_demo['f_labour_unemployed']/delta_demo['f_labour_unemployed'].sum())*delta_demo['delta_A']

delta_demo['B_pop_2016'] = (delta_demo['f_pop_2016']/delta_demo['f_pop_2016'].sum())*delta_demo['delta_B']
delta_demo['B_vm_minority'] = (delta_demo['f_vm_minority']/delta_demo['f_vm_minority'].sum())*delta_demo['delta_B']
delta_demo['B_income_lim'] = (delta_demo['f_income_lim']/delta_demo['f_income_lim'].sum())*delta_demo['delta_B']
delta_demo['B_labour_unemployed'] = (delta_demo['f_labour_unemployed']/delta_demo['f_labour_unemployed'].sum())*delta_demo['delta_B']

delta_demo['C_pop_2016'] = (delta_demo['f_pop_2016']/delta_demo['f_pop_2016'].sum())*delta_demo['delta_C']
delta_demo['C_vm_minority'] = (delta_demo['f_vm_minority']/delta_demo['f_vm_minority'].sum())*delta_demo['delta_C']
delta_demo['C_income_lim'] = (delta_demo['f_income_lim']/delta_demo['f_income_lim'].sum())*delta_demo['delta_C']
delta_demo['C_labour_unemployed'] = (delta_demo['f_labour_unemployed']/delta_demo['f_labour_unemployed'].sum())*delta_demo['delta_C']

Finally, we assemble our data into a plottable form and generate plots.

In [18]:
to_plot = delta_demo[[
    'A_pop_2016', 'B_pop_2016', 'C_pop_2016', 
    'A_vm_minority', 'B_vm_minority', 'C_vm_minority', 
    'A_income_lim', 'B_income_lim', 'C_income_lim',
    'A_labour_unemployed', 'B_labour_unemployed', 'C_labour_unemployed'
    ]]

pretty_names = {'income_lim': "Low Income (LIM)", 'pop_2016': 'Total Population', 'vm_minority': "Visible Minority", 'labour_unemployed':"Unemployed"}
melted = to_plot.melt().groupby('variable', as_index=False).sum()
melted['scenario'] = melted.variable.str[0]
melted['demographic'] = melted.variable.str[2:]
melted['demo_name'] = melted.demographic.map(pretty_names)
melted.head()

,variable,value,scenario,demographic,demo_name
0,A_income_lim,34.844541,A,income_lim,Low Income (LIM)
1,A_labour_unemployed,34.532953,A,labour_unemployed,Unemployed
2,A_pop_2016,34.919018,A,pop_2016,Total Population
3,A_vm_minority,45.766372,A,vm_minority,Visible Minority
4,B_income_lim,10.212393,B,income_lim,Low Income (LIM)


In [20]:
alt.Chart(melted).mark_bar().encode(
    alt.Y('scenario:N', title=None),
    alt.X('value:Q', title='Average Travel Time Savings (min)'),
    alt.Color('scenario:N', title='Scenario'),
    alt.Row('demo_name:N', title='', sort=['Total Population'], spacing=35)
).properties(
    title="Average Additional Transit Trips Induced for SmartTrack Scenarios",
    width=600,
    height=80
).configure(font='Roboto').configure_axis(grid=False).configure_view(strokeWidth=0).configure_title(fontSize=18)

alt.Chart(...)